In [2]:
import pandas as pd
import numpy as np
import random 

# Additional methods

In [3]:
# Calculate distance between 2 points in Euclidean space
# Positions should be 3D (X,Y,Z)

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [4]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p): 
    neighbours = []
    
    for i in range(len(positions[0])):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [5]:
# Average distance between neighbours

def distNeighbours():
    distances = []
    for i in range(len(positions[0])):
        neighbours = get_neighbours(i)
        for j in neighbours:
            list = get_dist(i, neighbours)
        summary = 0
        for x in list:
            summary += sum(x[1])
        distances.append(summary/len(list))
    
    avgNeighbourDist = sum(distances) / len(distances)
    return avgNeighbourDist

In [6]:
# Average difference between neighbour distances

def differenceNeighbourDist():
    distances = []
    for i in range(len(positions[0])):
        neighbours = get_neighbours(i)
        for j in neighbours:
            list = get_dist(i, neighbours)

        iter = 0
        for j in range(len(list)):
            cnt = 0
            for z in range(len(list)-1):
                cnt += abs(sum(list[j][1]) - sum(list[z+1][1]))
                iter += 1
        distances.append(cnt / iter)
    
    avgNeighbourDistDifferences = sum(distances)/len(distances)
    return avgNeighbourDistDifferences

In [7]:
# Add a key-value pair to the existing dictionary

def add_element(dict, key, value):
    if key not in dict:
        dict[key] = []
    dict[key].append(value)

In [8]:
# Returns with a dictionary. Key is the number of the node and 
# the value is the distant between the current node(act) and its neighbour.
# The list is ascendant by the distance.

def get_dist(act, neighbours):
    dist = {}
    
    for i in neighbours:
        add_element(dist, i, Euclidean_dist(act, i))
    
    dist = sorted(dist.items(), key=lambda x: x[1])
    
    return dist

In [9]:
# Calculates the full lenght of the path through the nodes.

def path_dist(g):
    dist = 0
    
    for i in range(len(g)-1):
        dist = dist + Euclidean_dist(g[i], g[i+1])
    
    return dist

In [10]:
# Entropy by numpy library

def entropy(labels, base=None):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0
    
    value, counts = np.unique(labels, return_counts=True)
    probs = counts/n_labels
    n_classes = np.count_nonzero(probs)
    
    if n_classes <= 1:
        return 0
    
    ent = 0.
    
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)
        
    return ent

In [11]:
# Gives back the distance between two chosen points

def distCalculator(a,b,dist):
    x = a * len(positions[0]) + b
    return np.sum(dist[x])

In [12]:
# ArrayList[num, [a,b]]

def getList(addList):
    a_b = []
    for i in range(len(positions[0])):
        for j in range(len(positions[0])):
            if j!=i:
                current = [i, j]
                a_b.append(current)
                
    list = []
    for i in range(len(a_b)):
        list.append([addList[i], a_b[i]])
    
    return list

In [13]:
# Get the differences of a node

def getNodeDiff(num, list):
    l = []
    for i in list:
        if i[1][0] == num:
            y = i[0]
            x = i[1][1]
            vec = [x,y]
            l.append(vec)
        l = sorted(l, key=lambda x: x[0])
    return l

In [14]:
# Calculates the average entropy for each routing tables

def avgEntropy(list):
    avg = []
    for i in range(len(positions[0])):
        node = getNodeDiff(i, list)
        num = 0
        for j in node:
            num += j[1]
        avg.append(num/len(positions[0]))
    return avg

# Pathfinding algorithms

## Greedy algorithm

In [15]:
# The algorithm chooses the neighbour node, which one is the closest the to end node.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy(p, z, routing_nodes, routing_dist):
    ready = False
    routing_nodes.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # Calculates the distances of the neighbour nodes from the end node
        dist = get_dist(z, neighbours)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            routing_nodes.append(z)
            ready=True
        
        # if 'act' has no neighbours 
        elif len(neighbours) == 0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            act = neighbours[0]
            routing_nodes.append(act)
            
        elif len(neighbours) > 1:
            not_found = False
            for i in dist:
                if i[0] not in routing_nodes:
                    routing_dist.append(Euclidean_dist(act, i[0]))
                    act = i[0]
                    routing_nodes.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                routing_dist.append(Euclidean_dist(act, dist[0][0]))
                act = dist[0][0]
                routing_nodes.append(act)

## Dijkstra algorithm

In [16]:
# The algorithm chooses the neighbour node, through the distance to the end node from 'p' is the shortest.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def dijkstra(p, z, routing_nodes, routing_dist):
    ready = False
    routing_nodes.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # Calculates the distances of the neighbour nodes from the end node
        dist = get_dist(z, neighbours)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            routing_nodes.append(z)
            ready=True
        
        # if 'act' has no neighbours 
        elif len(neighbours)==0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            act = neighbours[0]
            routing_nodes.append(act)
            
        elif len(neighbours) > 1:
            # distance from 'p' to the neighbours
            dist1 = get_dist(act, neighbours)
            # distance from the neighbours to the end node
            dist2 = get_dist(z, neighbours)
            dist = {}
            
            # Calculate the distance from 'p' through the neighbours to the end node
            for i in range(len(neighbours)):
                for j in range(len(neighbours)):
                    if dist1[i][0]==dist2[j][0]:
                        val1 = dist1[i][1]
                        val2 = dist2[j][1]
                        add_element(dist, dist1[i][0], val1[0]+val2[0])
            dist = sorted(dist.items(), key=lambda x: x[1])
            
            not_found = False
            for i in dist:
                if i[0] not in routing_nodes:
                    routing_dist.append(Euclidean_dist(act, i[0]))
                    act = i[0]
                    routing_nodes.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                routing_dist.append(Euclidean_dist(act, dist[0][0]))
                act = dist[0][0]
                routing_nodes.append(act)

## Random algorithm 1. (Fully random)

In [17]:
# The algorithm chooses randomly amongst neighbours

def rand(p, z, routing_nodes, routing_dist):
    ready = False
    routing_nodes.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # Calculates the distances of the neighbour nodes from the end node
        dist = get_dist(z, neighbours)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            routing_nodes.append(z)
            ready=True
        
        # if 'act' has no neighbours 
        elif len(neighbours) == 0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            act = neighbours[0]
            routing_nodes.append(act)
            
        elif len(neighbours) > 1:
            nextHop = random.choice(neighbours)
            print(nextHop)
            routing_dist.append(Euclidean_dist(act, nextHop))
            act = nextHop
            routing_nodes.append(act)

## Random algorithm 2. (Random decision with boundary) 

In [18]:
# The algorithm chooses randomly amongst neighbours, which ones are the same or within distance with boundary from the endpoint

def randBoundary(p, z, routing_nodes, routing_dist, boundary=0):
    ready = False
    routing_nodes.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # Calculates the distances of the neighbour nodes from the end node
        dist = get_dist(z, neighbours)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            routing_nodes.append(z)
            ready=True
        
        # if 'act' has no neighbours 
        elif len(neighbours) == 0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            act = neighbours[0]
            routing_nodes.append(act)
            
        elif len(neighbours) > 1:
            list = []
            for i in dist:
                if len(list) == 0:
                    list.append(i)
                elif sum(list[0][1]) + boundary >= i[1]:
                    list.append(i)
            reduced_neighbours = []
            for i in list:
                reduced_neighbours.append(i[0])
            nextHop = random.choice(reduced_neighbours)
            routing_dist.append(Euclidean_dist(act, nextHop))
            act = nextHop
            routing_nodes.append(act)

In [19]:
# The algorithm chooses the node which one is the closest to the current node and they are neighbours.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy_alternative(p, z, strech):
    ready = False
    routing = []
    routing.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing.append(z)
            return routing
        
        # if 'act' has no neighbours 
        elif len(neighbours)==0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            act = neighbours[0]
            routing.append(act)
            
        elif len(neighbours) > 1:
            dist = get_dist(act, neighbours)
            
            # print("Neighbours: ", neighbours)
            # print("DIST: ", dist)
            # print()
            
            not_found = False
            for i in dist:
                if i[0] not in routing:
                    act = i[0]
                    routing.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                act = dist[0][0]
                routing.append(act)

# Additional methods to the algorithms

### Adding elements to routing tables

In [20]:
# Add the path of p->z to the routing table 

def greedyAlgorithm(p,z,dist,nodes):
    routing_nodes = []
    routing_dist = []
    greedy(p,z,routing_nodes,routing_dist)
    if routing_nodes not in nodes:
        nodes.append(routing_nodes)
        dist.append(routing_dist)

In [21]:
# Add the path of p->z to the routing table 

def dijkstraAlgorithm(p,z,dist,nodes):
    routing_nodes = []
    routing_dist = []
    dijkstra(p,z,routing_nodes,routing_dist)
    if routing_nodes not in nodes:
        nodes.append(routing_nodes)
        dist.append(routing_dist)

In [22]:
# Add the path of p->z to the routing table 

def randAlgorithm(p,z,dist,nodes):
    routing_nodes = []
    routing_dist = []
    rand(p,z,routing_nodes,routing_dist)
    if routing_nodes not in nodes:
        nodes.append(routing_nodes)
        dist.append(routing_dist)

In [23]:
# Add the path of p->z to the routing table 

def randBoundaryAlgorithm(p, z, dist, nodes, boundary=0):
    routing_nodes = []
    routing_dist = []
    randBoundary(p,z,routing_nodes,routing_dist,boundary)
    if routing_nodes not in nodes:
        nodes.append(routing_nodes)
        dist.append(routing_dist)

In [24]:
# Add the path of p->z to the routing table 

def greedyAlternative(p,z,nodes):
    path = greedy_alternative(p,z)
    if path not in nodes:
        nodes.append(path)

### Iteration

In [25]:
# Iterate throught all of the nodes and run the greedy algorithm on them.

def greedyIter(dist, nodes):
    for i in range(len(positions[0])):
        for j in range(len(positions[0])):
            if i!= j:
                greedyAlgorithm(i,j,dist,nodes)

In [26]:
# Iterate throught all of the nodes and run the dijkstra algorithm on them.

def dijkstraIter(dist, nodes):
    for i in range(len(positions[0])):
        for j in range(len(positions[0])):
            if i!= j:
                dijkstraAlgorithm(i,j,dist,nodes)

In [27]:
# Iterate throught all of the nodes and run the rand algorithm on them.

def randIter(dist, nodes):
    for i in range(len(positions[0])):
        for j in range(len(positions[0])):
            if i!= j:
                randAlgorithm(i,j,dist,nodes)

In [28]:
# Iterate throught all of the nodes and run the rand algorithm on them.

def randBoundaryIter(dist, nodes, boundary=0):
    for i in range(len(positions[0])):
        for j in range(len(positions[0])):
            if i!= j:
                randBoundaryAlgorithm(i,j,dist,nodes,boundary)

In [29]:
# Iterate throught all of the nodes and run the greedy_alternative algorithm on them.

def greedyAlternativeIter():
    for i in range(len(positions[0])):
        for j in range(len(positions[0])):
            if i!= j:
                greedyAlternative(i,j)

# I/O methods

In [30]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [31]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [32]:
# for one col csv

def csv2list(name):
    df = pd.read_csv(name + ".csv", header=None)
    list=df[0].tolist()
    return list

In [33]:
# for multicol csv

def csv2array(name):
    df = pd.read_csv(name + ".csv", sep=';',header=None)
    array = df.values
    
    list = []
    for i in array:
        list.append(i[np.logical_not(np.isnan(i))])

    values = []
    for i in list:
        current = []
        for j in i:
            current.append(j)
        values.append(current)
        
    return values

In [34]:
# Print out the 'list' to the 'name.csv' file 

def printOut(name, list):
    with open(name + '.csv', 'w', newline='') as csv_1:
        csv_out = csv.writer(csv_1)
        csv_out.writerows([list[index]] for index in range(0, len(list)))

# Important variables, lists...

In [35]:
# List of the positions of nodes
positions = positionRead("Network/Brain_data/Brain1Positions")

# List of how nodes connected to each other
connections = connectionRead("Network/Brain_data/Brain1Connections")

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
